<a href="https://colab.research.google.com/github/sharjeelnawaz8182/Severstal-Steel-Defect-Detection-compitition/blob/main/Severstal_Steel_Defect_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import cv2
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
def kuwahara_normal_filter(image,kernel=7):
    pad_size = kernel//2
    
    height, width, channel = image.shape
    out_image = image.copy()
    
    pad_image = np.zeros((height+pad_size*2,width+pad_size*2,channel))
    for c in range(channel):
        pad_image[:,:,c] = np.pad(out_image[:,:,c],[pad_size,pad_size],'constant')
        
    for h in range(height):
        for w in range(width):
            for c in range(channel):
                #identify the area 1,2,3,4 range
                #in pad image
                cur_point_index = (h + pad_size,w + pad_size,c)
                area = np.zeros((4,kernel//2 +1, kernel//2 +1))
                
                area[0] = pad_image[h:(cur_point_index[0]+1),w:(cur_point_index[1]+1),c]
                area[1] = pad_image[h:(cur_point_index[0]+1),cur_point_index[1]:(cur_point_index[1]+pad_size+1),c]
                area[2] = pad_image[cur_point_index[0]:(cur_point_index[0]+1+pad_size),w:(cur_point_index[1]+1),c]
                area[3] = pad_image[cur_point_index[0]:(cur_point_index[0]+1+pad_size),cur_point_index[1]:(cur_point_index[1]+1+pad_size),c]
                
                
                
                std_area = [np.std(area[0]),np.std(area[1]),np.std(area[2]),np.std(area[3])]
                min_std_area_index = np.argwhere(std_area==np.min(std_area))[0,0]
             
                out_image[h,w,c] = np.sum(area[min_std_area_index])/(len(area[min_std_area_index])**2)
    return out_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_path = "/content/drive/MyDrive/compitition/severstal-steel-defect-detection"
import tensorflow as tf

def loadImages(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, 'train_images', file)
                          for file in os.listdir(path + "/train_images")
                          if file.endswith('.jpg')])
    return image_files
def loadImagest(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, '1', file)
                          for file in os.listdir(path + "/1")
                          if file.endswith('.jpg')])
    return image_files

In [ ]:
def display_one(a, title1 = "Original"):
    plt.imshow(a), plt.title(title1)
    plt.show()

In [ ]:
dataset=loadImages(image_path)

In [ ]:
dataset[4]

In [ ]:
train_df_original = pd.read_csv("/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train.csv")

In [ ]:
train_df_original.head()

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images/'
from glob import glob
train_fns = pd.Series(sorted(glob(TRAIN_PATH + '*.jpg')))
# train_fns.values
train_fns_split = train_fns.str.split('/', expand=True)
train_fns_split.head()

In [ ]:
from tqdm import tqdm
train_df = pd.DataFrame(columns=['ImageId_ClassId','EncodedPixels'])
# train_df.head()
for i in tqdm(range(len(train_fns_split))):
    for j in range(4):
        tmp_se = pd.Series( [train_fns_split[7][i]+'_{}'.format(j+1),None], index=train_df.columns )
        train_df = train_df.append( tmp_se, ignore_index=True )
train_df.head()

In [ ]:
for i in tqdm(range(len(train_df_original))):
    imageid_classid = train_df_original['ImageId'][i]+'_{}'.format(train_df_original['ClassId'][i])
    idx=train_df.query('ImageId_ClassId == "{}"'.format(imageid_classid)).index[0]
    train_df['EncodedPixels'][idx] = train_df_original['EncodedPixels'][i]
# print(train_df.shape)
train_df.head()

In [ ]:
train_df['image_id']=train_df['ImageId_ClassId'].apply(lambda x:x.split('_')[0])
train_df['class_id']=train_df['ImageId_ClassId'].apply(lambda x:x.split('_')[1])
train_df['has_defect'] = ~ train_df['EncodedPixels'].isna()

In [ ]:
train_df.head()

In [ ]:
count_defect=train_df.groupby('image_id').agg(np.sum).reset_index()
count_defect.sort_values('has_defect', ascending=False, inplace=True)
# print(mask_count_df.shape)
count_defect.head()

In [ ]:
sub_df_original=pd.read_csv("/content/drive/MyDrive/compitition/severstal-steel-defect-detection/sample_submission.csv")

In [ ]:
sub_df = pd.DataFrame(columns=['ImageId_ClassId','EncodedPixels'])
for i in tqdm(range(len(sub_df_original))):
    for j in range(4):
        tmp_se = pd.Series( [sub_df_original['ImageId'][i]+'_{}'.format(j+1),None], index=sub_df.columns )
        sub_df = sub_df.append( tmp_se, ignore_index=True )
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
sub_df.head()

In [ ]:
len(train_df_original)

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def rle2mask(mask_rle, shape=(256,1600)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1],dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (width, height))
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

In [ ]:
def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
pixel=np.ones((3,2))
pixel=pixel.flatten()

In [ ]:
train_df.head(40)

In [ ]:
sample_filename = '0002cc93b.jpg'
sample_image_df = train_df[train_df['image_id'] == sample_filename]
sample_path ="/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images/{sample_image_df['image_id'].iloc[0]}"
im = cv2.imread(sample_path)
sample_rles = sample_image_df['EncodedPixels'].values
sample_masks = build_masks(sample_rles, input_shape=(256, 1600))

for i in range(4):
    display_one(sample_masks[:, :, i])
    print(i)



In [ ]:
def __load_grayscale():
        image_path = "/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images"
        image_files = sorted([os.path.join(image_path, file)
                          for file in os.listdir(image_path)
                          if file.endswith('.jpg')])
        for i, sample in enumerate(image_files):
            item=sample
            if(item.split('/')[7]==sample_image_df['image_id'].iloc[0]):
              img = cv2.imread(sample, cv2.IMREAD_GRAYSCALE)
              img = img.astype(np.float32) / 255.
              img = np.expand_dims(img, axis=-1)

        return img

In [ ]:
from google.colab.patches import cv2_imshow
im=__load_grayscale()
cv2_imshow(im)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path="/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images"
                 ,batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['image_id'].iloc[ID]
            img_path = "{self.base_path}/{im_name}"
            img = self.__load_grayscale(im_name)
            
            # Store samples
            X[i,] = img

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['image_id'].iloc[ID]
            image_df = self.target_df[self.target_df['image_id'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y.astype(np.float32)
    
    def __load_grayscale(self, image_name):
        image_path = "/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images"
        image_files = sorted([os.path.join(image_path, file)
                          for file in os.listdir(image_path)
                          if file.endswith('.jpg')])
        for i, sample in enumerate(image_files):
            item=sample
            if(item.split('/')[7]==image_name):
              img = cv2.imread(sample, cv2.IMREAD_GRAYSCALE)
              img = img.astype(np.float32) / 255.
              img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [ ]:
BATCH_SIZE = 16

train_idx, val_idx = train_test_split(
    count_defect.index, random_state=200, test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=count_defect,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

val_generator = DataGenerator(
    val_idx, 
    df=count_defect,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

In [ ]:
def build_model(input_shape):
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(64, (3, 3), activation='elu', padding='same') (p4)
    c5 = Conv2D(64, (3, 3), activation='elu', padding='same') (c5)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

    c55 = Conv2D(128, (3, 3), activation='elu', padding='same') (p5)
    c55 = Conv2D(128, (3, 3), activation='elu', padding='same') (c55)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55)
    u6 = concatenate([u6, c5])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)

    u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u71 = concatenate([u71, c4])
    c71 = Conv2D(32, (3, 3), activation='elu', padding='same') (u71)
    c61 = Conv2D(32, (3, 3), activation='elu', padding='same') (c71)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='elu', padding='same') (u9)
    c9 = Conv2D(8, (3, 3), activation='elu', padding='same') (c9)

    outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])
    
    return model

In [ ]:
model = build_model((256, 1600, 1))

In [ ]:
 checkpoint_path="/content/drive/MyDrive/compitition/severstal-steel-defect-detection/train_images/cp.ckpt"

In [ ]:
model.load_weights(checkpoint_path)


In [ ]:
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='val_dice_coef', 
    verbose=1, 
    save_weights_only=True,
    save_best_only=True,
    mode='auto'
)

history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=True,
    workers=1,
    epochs=10)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['dice_coef', 'val_dice_coef']].plot()